In [1]:
# TODO: Set environment dynamically 
enviro = 'dev'

import pandas as pd
if enviro != 'dev': 
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth

In [24]:
####'
# Set Latest File Names & data path 
###
current_rental_fn = 'Copy of RENTAL 2020.xlsx - Sheet1.csv'
previous_rental_fn = 'rental billing 2019.csv'
sdat_fn = 'SDAT-CAN-ref-202105.csv'
data_out = 'fixed_up_rental_billing_2020.csv'

# Data file path 
if enviro == 'dev':
    path = 'data/'
else: 
    path = '/content/drive/My Drive/pita 2021/'

In [3]:
if enviro != 'dev':
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
df = pd.read_csv(path+current_rental_fn)
df = df[df['Customer #'].notna()]
df = df.rename(columns={'Customer #':'Customer Id','Dist/Account No    ':'Dist/Account No'})
len(df)

1528

In [5]:
df2 = pd.read_csv(path+previous_rental_fn)

In [6]:
df2 = df2[df2['Customer Id'].notna()]
df2['License Id'] = df2.apply(lambda x: x['License Id'].replace('19-','20-'),axis=1)

In [7]:
#df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "both"')a
df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "both"')


,Customer Id,Name,Name 2,Address 2,State,Zip,License Id,License Type,Property Location,Unnamed: 9,Unnamed: 10,RENTAL,Dist/Account No,_merge
0,RR-07388,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00001,RENTAL,416 BOUNDARY AVE 128,True,True,1.0,07- 113935,both
1,RR-07981,DAGOSTINO COREY,C/O WRIGHT PROP MANAGEMEMENT,,MD,21643,20-00002,RENTAL,704 CHURCH ST 374,True,True,2.0,07- 148038,both
2,RR-04889,JAMES INVESTMENTS LLC,,,MD,21613,20-00003,RENTAL,809 PHILLIPS ST 1425,True,True,1.0,07- 164289,both
3,RR-00375,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00004,RENTAL,715 PEACHBLOSSOM AVE 1370,True,True,1.0,07- 130538,both
4,RR-00380,RUDDY DUCK LLC,,,MD,21613,20-00005,RENTAL,727 PEACHBLOSSOM AVE 1375,True,True,1.0,07- 126905,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,RR-00004,MCB RENTALS,,,MD,21613,20-01544,RENTAL,307 CHOPTANK AVE 348,True,True,1.0,,both
1519,RR-09110,AKARDAS IBRAHIM & LISA,,,PA,15237,20-01545,RENTAL,306 OLD SQUAW CT 7535,True,True,1.0,,both
1520,RR-09665,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01546,RENTAL,334 APPLEBY SCHOOL RD,True,True,1.0,07- 215096,both
1521,RR-09639,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01547,RENTAL,338 APPLEBY SCHOOL RD 8024,True,True,1.0,,both


In [8]:
no_longer_rented = df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "right_only"')
no_longer_rented = no_longer_rented.rename(columns={'Customer #':'Customer Id','Dist/Account No    ':'Dist/Account No'})
len(no_longer_rented)

27

In [9]:
new_rentals = df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "left_only"')
new_rentals = new_rentals.rename(columns={'Customer #':'Customer Id','Dist/Account No    ':'Dist/Account No'})
len(new_rentals)

5

In [10]:
# fix up account numbers to make apn-format tax acctid column
no_longer_rented['Dist/Account No'] = no_longer_rented['Dist/Account No'].fillna('-1')
no_longer_rented['Dist/Account No'] = no_longer_rented.apply(lambda x: x['Dist/Account No'].replace(' ',''),axis=1)
no_longer_rented['acctid'] = no_longer_rented.apply(lambda x: "10{}".format(x['Dist/Account No'].replace('-','')), axis = 1)

In [11]:
sdat = pd.read_csv(path+sdat_fn)
sdat.acctid = sdat.acctid.apply(lambda x: str(x).strip())
sdat = sdat.set_index('acctid')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
sdat.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 23191 entries, 1001000012 to 1018001837
Data columns (total 142 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   jurscode                                                  object 
 1   digxcord                                                  float64
 2   digycord                                                  float64
 3   ct2010                                                    float64
 4   bg2010                                                    float64
 5   geogcode                                                  int64  
 6   ooi                                                       object 
 7   resityp                                                   object 
 8   address                                                   object 
 9   strtnum                                                   float64
 10  strtdir                 

In [19]:
no_longer_rented[['acctid','RENTAL  ']].merge(sdat,on='acctid',how='outer',indicator=True).query('_merge == "both"')[['acctid','RENTAL  ','address','ownname1','ownname2']]
#.to_csv('/content/drive/My Drive/pita 2021/no_longer_rented-2020.csv')

,acctid,RENTAL,address,ownname1,ownname2
0,1007143230,1.0,406 KENT ST,MILLER DEREK E,NaN
1,1007213905,1.0,700 CATTAIL COVE,REFOSCO JAMES,REFOSCO TOBY
2,1007121725,2.0,307 BYRN ST,SPEAR ASHLY BRANDEN ETAL,SPEAR ASHLY B. ETAL
3,1007138423,1.0,414 GLENBURN AVE,THOMAS CYNTHIA E G,NaN
4,1007149387,1.0,908 PEACHBLOSSOM AVE,RUDDY DUCK LLC,NaN
5,1007138695,1.0,713 HUGHLETT ST,MADISON BAY HOLDINGS LLC,NaN
6,1007163169,1.0,638 WASHINGTON ST,MADISON BAY HOLDINGS LLC,NaN
7,1007149034,1.0,327 CEMETERY AVE,COALE FAMILY INVESTMENT COMPANY,LLC THE
8,1007111134,1.0,705 GOLDSBOROUGH AVE,BRAMBLE RUTH L,NaN
9,1007112009,3.0,934 PINE ST,CARLTON COURT ASSOCIATES LIMITED,PARTNERSHIP


In [20]:
# fix up account numbers to make apn-format tax acctid column
new_rentals['Dist/Account No'] = new_rentals['Dist/Account No'].fillna('-1')
new_rentals['Dist/Account No'] = new_rentals.apply(lambda x: x['Dist/Account No'].replace(' ',''),axis=1)
new_rentals['acctid'] = new_rentals.apply(lambda x: "10{}".format(x['Dist/Account No'].replace('-','')), axis = 1)


In [21]:
def cleanup_address(a):
  b = a.strip().replace('.','').replace('AVENUE','AVE')
  pieces = b.split()
  if pieces[-1][0].isnumeric():
    return " ".join(pieces[0:-1]).replace('.','')
  else:
    return b

#new_rentals['Property Location'] = 
new_rentals['Property Location'] = new_rentals.apply(lambda x: cleanup_address( x['Property Location'] ), axis=1)
new_rentals[['Property Location']].merge(sdat.reset_index(),left_on='Property Location',right_on='address',how='outer',indicator=True).query('_merge == "both"')[['acctid','address','ownname1','ownname2']]#.to_csv('/content/drive/My Drive/pita 2021/new_rentals-2020.csv')

,acctid,address,ownname1,ownname2
0,1007141947,911 RACE ST,TURNER JUSTIN B,WALLACE HANNAH
1,1007167202,512 GAY ST,KHAREL CHHATRAMANI,KHAREL SARITA KARKI
2,1007120060,800 RACE ST,HALCYON ASSETS LLC,NaN
3,1007106122,401 LIGHT ST,MCCORMICK 103 LLC,NaN
4,1007124236,627 DOUGLAS ST,CEPHAS GILBERT C,NaN


In [ ]:
#.to_csv('/content/drive/My Drive/pita 2021/new_rentals-2020.csv')

In [25]:
print('useful rows:',len(df))
print('Updated column headers')
print(df.columns)

patched = df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "both"')
patched.rename(columns={'Dist/Account No    ':'Dist/Account No','Property Location':'address'}, inplace=True)
patched['Dist/Account No'] = patched['Dist/Account No'].fillna('-1')
patched['Dist/Account No'] = patched.apply(lambda x: x['Dist/Account No'].replace(' ',''),axis=1)
patched['acctid'] = patched.apply(lambda x: "10{}".format(x['Dist/Account No'].replace('-','')), axis = 1)
patched = patched.set_index('acctid')
patched = patched.drop(columns=['_merge'])

#patched.drop(columns=['_merge']).merge(sdat.reset_index(),left_on='Property Location',right_on='address',how='outer',indicator=True).query('_merge == "left_only"')[['acctid','address','ownname1','ownname2']]
patched.to_csv(path+data_out)
print(patched.columns)
patched

useful rows: 1528
Updated column headers
Index(['Customer Id', 'Name', 'Name 2', 'Address 2', 'State', 'Zip',
       'License Id', 'License Type', 'Property Location', 'Unnamed: 9',
       'Unnamed: 10'],
      dtype='object')
Index(['Customer Id', 'Name', 'Name 2', 'Address 2', 'State', 'Zip',
       'License Id', 'License Type', 'address', 'Unnamed: 9', 'Unnamed: 10',
       'RENTAL  ', 'Dist/Account No'],
      dtype='object')


,Customer Id,Name,Name 2,Address 2,State,Zip,License Id,License Type,address,Unnamed: 9,Unnamed: 10,RENTAL,Dist/Account No
acctid,,,,,,,,,,,,,
1007113935,RR-07388,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00001,RENTAL,416 BOUNDARY AVE 128,True,True,1.0,07-113935
1007148038,RR-07981,DAGOSTINO COREY,C/O WRIGHT PROP MANAGEMEMENT,,MD,21643,20-00002,RENTAL,704 CHURCH ST 374,True,True,2.0,07-148038
1007164289,RR-04889,JAMES INVESTMENTS LLC,,,MD,21613,20-00003,RENTAL,809 PHILLIPS ST 1425,True,True,1.0,07-164289
1007130538,RR-00375,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00004,RENTAL,715 PEACHBLOSSOM AVE 1370,True,True,1.0,07-130538
1007126905,RR-00380,RUDDY DUCK LLC,,,MD,21613,20-00005,RENTAL,727 PEACHBLOSSOM AVE 1375,True,True,1.0,07-126905
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,RR-00004,MCB RENTALS,,,MD,21613,20-01544,RENTAL,307 CHOPTANK AVE 348,True,True,1.0,
10,RR-09110,AKARDAS IBRAHIM & LISA,,,PA,15237,20-01545,RENTAL,306 OLD SQUAW CT 7535,True,True,1.0,
1007215096,RR-09665,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01546,RENTAL,334 APPLEBY SCHOOL RD,True,True,1.0,07-215096


In [26]:
df.merge(df2[['License Id','RENTAL  ','Dist/Account No    ']],on='License Id',how='outer',indicator=True).query('_merge == "both"')

,Customer Id,Name,Name 2,Address 2,State,Zip,License Id,License Type,Property Location,Unnamed: 9,Unnamed: 10,RENTAL,Dist/Account No,_merge
0,RR-07388,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00001,RENTAL,416 BOUNDARY AVE 128,True,True,1.0,07- 113935,both
1,RR-07981,DAGOSTINO COREY,C/O WRIGHT PROP MANAGEMEMENT,,MD,21643,20-00002,RENTAL,704 CHURCH ST 374,True,True,2.0,07- 148038,both
2,RR-04889,JAMES INVESTMENTS LLC,,,MD,21613,20-00003,RENTAL,809 PHILLIPS ST 1425,True,True,1.0,07- 164289,both
3,RR-00375,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00004,RENTAL,715 PEACHBLOSSOM AVE 1370,True,True,1.0,07- 130538,both
4,RR-00380,RUDDY DUCK LLC,,,MD,21613,20-00005,RENTAL,727 PEACHBLOSSOM AVE 1375,True,True,1.0,07- 126905,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,RR-00004,MCB RENTALS,,,MD,21613,20-01544,RENTAL,307 CHOPTANK AVE 348,True,True,1.0,,both
1519,RR-09110,AKARDAS IBRAHIM & LISA,,,PA,15237,20-01545,RENTAL,306 OLD SQUAW CT 7535,True,True,1.0,,both
1520,RR-09665,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01546,RENTAL,334 APPLEBY SCHOOL RD,True,True,1.0,07- 215096,both
1521,RR-09639,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01547,RENTAL,338 APPLEBY SCHOOL RD 8024,True,True,1.0,,both


In [27]:
bad_patched = patched.reset_index().query('acctid == "10" or acctid == "1007"')
bad_patched
bad_patched['address'] = bad_patched.apply(lambda x: cleanup_address( x['address'] ), axis=1)
bad_fixed = bad_patched[['address']].merge(sdat.reset_index(),left_on='address',right_on='address',how='outer',indicator=True).query('_merge == "both"')[['acctid','address','ownname1','ownname2']]#.to_csv('/content/drive/My Drive/pita 2021/new_rentals-2020.csv')

In [28]:
for s in patched.reset_index().query('acctid != "10" or acctid != "1007"').acctid.unique():#.append(bad_fixed).acctid.unique():
  print(s)
#bad_fixed

1007113935
1007148038
1007164289
1007130538
1007126905
1007104294
1007148178
1007144180
1007151624
1007108796
1007109717
1007114036
1007119445
1007104731
1007135033
1007141238
1007106092
1007165323
1007164963
1007133111
1007161522
1007132409
1007106602
1007148976
1007167482
1007142536
1007178379
1007178565
1007192991
1007193068
1007192975
1007123299
1007137370
1007123310
1007121814
1007106920
1007123191
1007166095
1007147856
1007164874
1007125100
1007144679
1007139055
1007104057
1007172508
1007120540
1007108672
1007153619
10
1007165560
1007165579
1007165498
1007165528
1007165536
1007165544
1007165552
1007122519
1007148828
1007145144
1007163533
1007147597
1007130791
1007122535
1007144830
1007113420
1007107412
1007145802
1007109067
1007146914
1007128819
1007141343
1007110529
1007170513
1007134274
1007113641
1007124449
1007147465
1007147473
1007125305
1007137214
1007126247
1007122225
1007110421
1007145470
1007143397
1007105053
1007142668
1007106076
1007121342
1007144105
1007139438
1007108